In [1]:
import pandas as pd 

In [4]:
ebayReport = pd.read_csv('cleanedEbaySalesData.csv', encoding = 'ISO-8859-1')

In [6]:
ebayReport.head()

,Type,Date,Num,Ship To Address 1,Ship To Address 2,Ship Postal Code,Item,Qty,Sales Price,Amount
0,Invoice,02/04/2019,14496,Dallin Young DDS,"114 Minnie Street,",99701,3MD-51102(E),4,30.00,120.00
1,Invoice,02/04/2019,14496,Dallin Young DDS,"114 Minnie Street,",99701,Shipping Charge,1,4.00,4.00
2,Credit Memo,02/04/2019,14500,Dallin Young DDS,"114 Minnie Street,",99701,3MD-51102(E),-4,30.00,-120.00
3,Credit Memo,02/04/2019,14500,Dallin Young DDS,"114 Minnie Street,",99701,Shipping Charge,-1,4.00,-4.00
4,Invoice,04/12/2019,17163,Glennallen Family Dentistry Dr. Kevin She,Mile 187 Glenn highway/mabel lane,99588,DEN-66002861,1,59.99,59.99


In [8]:
# Dropping an unnecessary column 
ebayReport = ebayReport.drop(columns = {'Sales Price'})

# Renaming a column to properly represent it's content
ebayReport = ebayReport.rename(columns = {'Amount':'Sales Total'})

# Dropping an unnecessary column 
ebayReport = ebayReport.drop(columns = {'Ship To Address 1'})

# Renaming a column to properly represent it's content
ebayReport = ebayReport.rename(columns = {'Ship To Address 2':'Address'})

# Renaming a column to properly represent it's content
ebayReport = ebayReport.rename(columns ={'Num':'Invoice Number'})

In [41]:
ebayReport = ebayReport.loc[ebayReport['Type']=='Invoice']

In [13]:
# from dfply import * 
# data= pd.read_csv('cleanedEbaySalesData.csv', encoding = 'ISO-8859-1')

# data >> select(X.Type, X.Date, X.Qty) >> mask(X.Type == "Invoice") >> head(3) 

# data >> group_by(X.Type) >> summarize(TotalQuantity = X.Qty.sum()) 

# data >> arrange(X.Date, ascending = False) >> head(5) 

In [43]:
ebayReport.head()

,Type,Date,Invoice Number,Address,Ship Postal Code,Item,Qty,Sales Total
0,Invoice,02/04/2019,14496,"114 Minnie Street,",99701,3MD-51102(E),4,120.00
1,Invoice,02/04/2019,14496,"114 Minnie Street,",99701,Shipping Charge,1,4.00
4,Invoice,04/12/2019,17163,Mile 187 Glenn highway/mabel lane,99588,DEN-66002861,1,59.99
5,Invoice,04/12/2019,17163,Mile 187 Glenn highway/mabel lane,99588,Shipping Charge,1,4.00
8,Invoice,01/28/2019,14050,412 E Pioneer Ave,99603,3MD-6021A3,3,170.85


In [50]:
# In this cell, i wil be adding the main IDs first and then landing costs to each item by importing the 'databaseVersion1' 
databaseLandingCosts = pd.read_csv('net32CostOfGoods.csv')

In [53]:
databaseLandingCosts.head()

,Item,Description,Cost
0,3MD-10771(E),3M ESPE IMPRINT 3 HEAVY BODY REFILL CARTRIDGES...,76.67
1,3MD-10771J,3M ESPE IMPRINT 3 HEAVY BODY CARTRIDGE JUMBO P...,298.09
2,3MD-12627(E),3M ESPE CLINPRO SEALANT SYRINGE REFILL 10 X1.2ML,10.66
3,3MD-1942FB,3M ESPE MASKS FLUID RESIST (50/BX),18.11
4,3MD-1942FR,3M ESPE MASKS FLUID RESIST (50/BX),18.11


In [ ]:
# Adding in the landing costs
ebayReport['landing cost'] = '0'

for i in range(0,len(ebayReport['Item'].unique())):
    for j in range(0,len(databaseLandingCosts['Item'].unique())):
        if ebayReport['Item'].unique()[i] == databaseLandingCosts['Item'][j]:
            ebayReport.loc[ebayReport['Item']==ebayReport['Item'].unique()[i],'landing cost'] = databaseLandingCosts['Cost'][j]

In [59]:
# ebayReportNoSC = ebayReport.loc[ebayReport['Item'] != 'Shipping Charge']
ebayReport.dtypes
ebayReport['landing cost'] = pd.to_numeric(ebayReport['landing cost'])

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
#ebayReport = ebayReport.reset_index()
ebayReport = ebayReport.drop(columns = {'index'})
ebayReport.head()

,Type,Date,Invoice Number,Address,Ship Postal Code,Item,Qty,Sales Total,landing cost
0,Invoice,02/04/2019,14496,"114 Minnie Street,",99701,3MD-51102(E),4,120.00,97.12
1,Invoice,02/04/2019,14496,"114 Minnie Street,",99701,Shipping Charge,1,4.00,0.00
2,Invoice,04/12/2019,17163,Mile 187 Glenn highway/mabel lane,99588,DEN-66002861,1,59.99,41.10
3,Invoice,04/12/2019,17163,Mile 187 Glenn highway/mabel lane,99588,Shipping Charge,1,4.00,0.00
4,Invoice,01/28/2019,14050,412 E Pioneer Ave,99603,3MD-6021A3,3,170.85,40.25


In [68]:
# In this cell i am adjusting the landing costs with respect to the amount of quantity bought from that record
for i in range(0,len(ebayReport['Item'])):
    ebayReport['landing cost'][i] = ebayReport['landing cost'][i]*ebayReport['Qty'][i]
    #print(ebayReport['landing cost'][i])
ebayReport

C:\Users\carlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Type,Date,Invoice Number,Address,Ship Postal Code,Item,Qty,Sales Total,landing cost
0,Invoice,02/04/2019,14496,"114 Minnie Street,",99701,3MD-51102(E),4,120.00,388.48
1,Invoice,02/04/2019,14496,"114 Minnie Street,",99701,Shipping Charge,1,4.00,0.00
2,Invoice,04/12/2019,17163,Mile 187 Glenn highway/mabel lane,99588,DEN-66002861,1,59.99,41.10
3,Invoice,04/12/2019,17163,Mile 187 Glenn highway/mabel lane,99588,Shipping Charge,1,4.00,0.00
4,Invoice,01/28/2019,14050,412 E Pioneer Ave,99603,3MD-6021A3,3,170.85,120.75
5,Invoice,01/28/2019,14050,412 E Pioneer Ave,99603,Shipping Charge,1,4.00,0.00
6,Invoice,01/03/2019,12968,605 Marine Ave,99611,3MD-5905A3,10,420.00,279.80
7,Invoice,01/03/2019,12968,605 Marine Ave,99611,3MD-5905A2,10,420.00,279.80
8,Invoice,07/31/2019,21947,"10767 KENAI SPUR HWY, STE A",99611-7836,3MD-51102(E),3,98.97,72.84
9,Invoice,07/31/2019,21947,"10767 KENAI SPUR HWY, STE A",99611-7836,Shipping Charge,1,37.64,0.00


In [69]:
ebayReport.to_csv(r"ebayPreAggReport.csv")#"")#ebayReport['landing cost'].unique()